In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from _ import *

In [ ]:
def get_self_mutual_information(ma):

    ma = ma.copy()

    assert 0 <= ma.min()

    sh = ma.shape

    de1 = 1 / sh[0]

    de2 = 1 / sh[1]

    ma += 1

    mar = ma / (ma.sum() * de1 * de2)

    marm = np.outer(mar.sum(axis=1) * de2, mar.sum(axis=0) * de1)

    return kwat.information.get_kld(mar, marm)

In [ ]:
def get_extremeselect(da, se, di, st):

    return da.loc[kwat.series.get_extreme(se, di, st=st), :]

In [ ]:
def isolateselect_by_median(da, di, st):

    if di == "<":

        da = da.clip(upper=0)

    elif di == ">":

        da = da.clip(lower=0)

    da = get_extremeselect(da, da.median(axis=1), di, st)

    if di == "<":

        da *= -1

    return da

In [ ]:
kwat.path.make(DIS)

In [ ]:
for ie, (da, na) in enumerate(zip(DAR_, (di["na"] for di in SE["dim_"]))):

    va, se, st = SE["si"]

    if va == "self_mutual_information":

        da = pd.DataFrame(
            data=get_self_mutual_information(da.values),
            index=da.index,
            columns=da.columns,
        )

    ti = "{} {}".format(na, va)

    kwat.feature_by_sample.summarize(da, title=ti)

    if se == "standard_deviation":

        da = get_extremeselect(da, da.std(axis=1), ">", st)

    else:

        da_ = [isolateselect_by_median(da, di, st) for di in ["<", ">"]]

        if se == "<>":

            da = pd.concat(da_)

            da.index = pd.Index(
                ["{}(-)".format(la) for la in da_[0].index.values]
                + ["{}(+)".format(la) for la in da_[1].index.values],
                name=da.index.name,
            )

        elif se == "<":

            da = da_[0]

        elif se == ">":

            da = da_[1]

    kwat.feature_by_sample.summarize(da, title="{} {} {}".format(ti, se, st))

    da.to_csv(path_or_buf=os.path.join(DIS, "{}.tsv".format(ie + 1)), sep="\t")